In [49]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
sns.set_context('paper')
# pd.set_option('display.max_columns', None)

In [50]:
with open("../Data/simu6b_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)
# df_study = df_study.loc[df_study.session < 500]
# df_test = df_test.loc[df_test.session < 500]

In [51]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,session
0,140,838,BLUEBERRY,LOCK,0
1,1587,1240,WARDROBE,SAXOPHONE,0
2,1132,702,PROTON,HELMET,0
3,299,1282,CHILD,SHELTER,0
4,1391,1059,STRAWBERRY,PICKLE,0
...,...,...,...,...,...
11995,301,1184,CHIMNEY,REPORT,999
11996,57,965,ATTIRE,NUT,999
11997,155,1112,BOULDER,POSSUM,999
11998,106,1368,BEAR,STAKE,999


In [52]:
df_test

,test_itemno1,test_itemno2,test_item,correct_ans,order,rep,test,session,test_pos,pair_idx
0,210,-1,CABBAGE,327,1,1,1,0,1,11
1,1251,-1,SCOTCH,819,2,1,1,0,2,8
2,145,-1,BODY,1198,1,1,1,0,3,7
3,1059,-1,PICKLE,1391,2,1,1,0,4,5
4,711,-1,HOE,1519,1,1,1,0,5,9
...,...,...,...,...,...,...,...,...,...,...
23995,965,-1,NUT,57,2,1,2,999,20,11997
23996,1075,-1,PISTOL,644,2,1,2,999,21,11993
23997,1090,-1,PLUM,1609,2,1,2,999,22,11991
23998,1184,-1,REPORT,301,2,1,2,999,23,11996


### Run CMR-IA

In [53]:
s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [54]:
params = cmr.make_default_params()
params.update(
    beta_enc = 0.5,
    beta_rec = 0.1, # beta for retrieved item
    beta_cue = 0.45, # beta for cue
    beta_distract = 0.1,
    gamma_fc = 0.35,
    gamma_cf = 0.35,
    s_fc = 0.1,
    s_cf = 0.1,
    c_thresh = 0.64,
    c_thresh_ass = 0.01,
    kappa = 0.02,
    lamb = 0.01,
    eta = 0.005,
    phi_s = 1,
    phi_d = 0.6,
    nitems_in_accumulator = 24,
    d_ass = 0.7,
    learn_while_retrieving = True,
    omega = 5,
    alpha = 1
)
params

{'beta_enc': 0.5,
 'beta_rec': 0.1,
 'beta_cue': 0.45,
 'beta_rec_post': 0.5,
 'beta_distract': 0.1,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.1,
 's_fc': 0.1,
 'kappa': 0.02,
 'eta': 0.005,
 'omega': 5,
 'alpha': 1,
 'c_thresh': 0.64,
 'c_thresh_ass': 0.01,
 'd_ass': 0.7,
 'lamb': 0.01,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 24,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.35,
 'gamma_cf': 0.35}

In [55]:
df_simu, f_in, f_dif = cmr.run_success_multi_sess(params, df_study, df_test, s_mat, mode='CR-CR')
df_simu['test_pos'] = np.tile(np.arange(1,25),1000)
df_simu

CMR2 Time: 27.582019329071045


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos
0,0,210,-1,327.0,1830.0,0.797872,1
1,0,1251,-1,-2.0,-2.0,0.627754,2
2,0,145,-1,-2.0,-2.0,0.554538,3
3,0,1059,-1,-2.0,-2.0,0.638699,4
4,0,711,-1,1198.0,2360.0,0.641787,5
...,...,...,...,...,...,...,...
23995,999,965,-1,57.0,1250.0,0.817659,20
23996,999,1075,-1,644.0,1010.0,0.845879,21
23997,999,1090,-1,-2.0,-2.0,0.544596,22
23998,999,1184,-1,301.0,1150.0,0.859984,23


In [56]:
df_simu = df_simu.merge(df_test,on=['session','test_itemno1','test_itemno2', 'test_pos'])
df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans
df_simu

,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct
0,0,210,-1,327.0,1830.0,0.797872,1,CABBAGE,327,1,1,1,11,True
1,0,1251,-1,-2.0,-2.0,0.627754,2,SCOTCH,819,2,1,1,8,False
2,0,145,-1,-2.0,-2.0,0.554538,3,BODY,1198,1,1,1,7,False
3,0,1059,-1,-2.0,-2.0,0.638699,4,PICKLE,1391,2,1,1,5,False
4,0,711,-1,1198.0,2360.0,0.641787,5,HOE,1519,1,1,1,9,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,999,965,-1,57.0,1250.0,0.817659,20,NUT,57,2,1,2,11997,True
23996,999,1075,-1,644.0,1010.0,0.845879,21,PISTOL,644,2,1,2,11993,True
23997,999,1090,-1,-2.0,-2.0,0.544596,22,PLUM,1609,2,1,2,11991,False
23998,999,1184,-1,301.0,1150.0,0.859984,23,REPORT,301,2,1,2,11996,True


In [57]:
df_cond = df_simu.groupby(["pair_idx","test"])['order'].mean().to_frame(name='corr_rate').reset_index()
df_cond = df_cond.pivot_table(index='pair_idx',columns='test',values='corr_rate').reset_index()
df_cond.columns = ['pair_idx','test1','test2']
def cond(x):
    test1 = x['test1']
    test2 = x['test2']
    if test1 == 1 and test2 == 1:
        return 'F-F'
    elif test1 == 1 and test2 == 2:
        return 'F-B'
    elif test1 == 2 and test2 == 1:
        return 'B-F'
    elif test1 == 2 and test2 == 2:
        return 'B-B'

df_cond['cond'] = df_cond.apply(lambda x:cond(x),axis=1)
df_cond['cong'] = df_cond.apply(lambda x: 'Identical' if x['cond'] == 'F-F' or x['cond'] == 'B-B' else 'Reversed',axis=1)
df_cond

,pair_idx,test1,test2,cond,cong
0,1,2.0,1.0,B-F,Reversed
1,2,1.0,2.0,F-B,Reversed
2,3,2.0,1.0,B-F,Reversed
3,4,1.0,1.0,F-F,Identical
4,5,2.0,2.0,B-B,Identical
...,...,...,...,...,...
11995,11996,2.0,2.0,B-B,Identical
11996,11997,1.0,2.0,F-B,Reversed
11997,11998,2.0,1.0,B-F,Reversed
11998,11999,1.0,1.0,F-F,Identical


In [58]:
pairidx2cond = df_cond.loc[:,['pair_idx','cond']].set_index("pair_idx").to_dict()['cond']
pairidx2cong = df_cond.loc[:,['pair_idx','cong']].set_index("pair_idx").to_dict()['cong']
df_simu['cond'] = df_simu.apply(lambda x:pairidx2cond[x['pair_idx']],axis=1)
df_simu['cong'] = df_simu.apply(lambda x:pairidx2cong[x['pair_idx']],axis=1)
df_simu.head(24)


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct,cond,cong
0,0,210,-1,327.0,1830.0,0.797872,1,CABBAGE,327,1,1,1,11,True,F-F,Identical
1,0,1251,-1,-2.0,-2.0,0.627754,2,SCOTCH,819,2,1,1,8,False,B-B,Identical
2,0,145,-1,-2.0,-2.0,0.554538,3,BODY,1198,1,1,1,7,False,F-F,Identical
3,0,1059,-1,-2.0,-2.0,0.638699,4,PICKLE,1391,2,1,1,5,False,B-B,Identical
4,0,711,-1,1198.0,2360.0,0.641787,5,HOE,1519,1,1,1,9,False,F-B,Reversed
5,0,838,-1,-2.0,-2.0,0.527408,6,LOCK,140,2,1,1,1,False,B-F,Reversed
6,0,153,-1,-2.0,-2.0,0.596206,7,BOTTLE,736,1,1,1,6,False,F-B,Reversed
7,0,1587,-1,-2.0,-2.0,0.575857,8,WARDROBE,1240,1,1,1,2,False,F-B,Reversed
8,0,856,-1,-2.0,-2.0,0.545230,9,MAID,607,2,1,1,10,False,B-B,Identical
9,0,702,-1,-2.0,-2.0,0.572660,10,HELMET,1132,2,1,1,3,False,B-F,Reversed


In [59]:
len(f_in[0])

24

In [60]:
sessions = np.unique(df_simu.session.to_numpy())
tmp_corr_fin = []
tmp_omean_fin = []
tmp_omax_fin = []

for sess in sessions:
    df_tmp = df_study.loc[df_study.session == sess]
    tmp1 = df_tmp.study_itemno1.to_numpy()
    tmp2 = df_tmp.study_itemno2.to_numpy()
    tmp = np.concatenate((tmp1, tmp2))
    tmp = np.sort(tmp)

    df_tmp1 = df_simu.loc[df_simu.session == sess]
    tmp_test = df_tmp1.test_itemno1
    tmp_corr = df_tmp1.correct_ans
    # tmp_test = df_simu.loc[df_simu.session == sess, 'test_itemno']
    # tmp_corr = df_simu.loc[df_simu.session == sess, 'correct_ans']
    testid = np.searchsorted(tmp, tmp_test)
    corrid = np.searchsorted(tmp, tmp_corr)

    fin_sess = f_dif[sess]
    # tmp_corr_fin = []
    # tmp_omean_fin = []
    # tmp_omax_fin = []
    for i in range(len(testid)):
        tmp_corr_fin.append(fin_sess[i][corrid[i]])
        tmp_omean_fin.append(np.mean(np.delete(fin_sess[i], corrid[i])))
        tmp_omax_fin.append(np.delete(fin_sess[i], corrid[i]).max())
    # df_simu.loc[df_simu.session == sess, 'corr_fin']= tmp_corr_fin
    # df_simu.loc[df_simu.session == sess, 'omean_fin'] = tmp_omean_fin
    # df_simu.loc[df_simu.session == sess, 'omax_fin'] = tmp_omax_fin

df_simu['corr_fin']= tmp_corr_fin
df_simu['omean_fin'] = tmp_omean_fin
df_simu['omax_fin'] = tmp_omax_fin
df_simu

,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct,cond,cong,corr_fin,omean_fin,omax_fin
0,0,210,-1,327.0,1830.0,0.797872,1,CABBAGE,327,1,1,1,11,True,F-F,Identical,-0.338427,-0.893156,-0.417645
1,0,1251,-1,-2.0,-2.0,0.627754,2,SCOTCH,819,2,1,1,8,False,B-B,Identical,-0.484410,-1.256948,-0.488696
2,0,145,-1,-2.0,-2.0,0.554538,3,BODY,1198,1,1,1,7,False,F-F,Identical,-0.446878,-1.460890,-0.498679
3,0,1059,-1,-2.0,-2.0,0.638699,4,PICKLE,1391,2,1,1,5,False,B-B,Identical,-0.464744,-1.659222,-0.479372
4,0,711,-1,1198.0,2360.0,0.641787,5,HOE,1519,1,1,1,9,False,F-B,Reversed,-0.455372,-1.882418,-0.477689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,999,965,-1,57.0,1250.0,0.817659,20,NUT,57,2,1,2,11997,True,F-B,Reversed,-0.010037,-3.078723,-0.241682
23996,999,1075,-1,644.0,1010.0,0.845879,21,PISTOL,644,2,1,2,11993,True,B-B,Identical,0.036203,-3.494364,-0.249831
23997,999,1090,-1,-2.0,-2.0,0.544596,22,PLUM,1609,2,1,2,11991,False,B-B,Identical,-0.460750,-3.922968,-0.315136
23998,999,1184,-1,301.0,1150.0,0.859984,23,REPORT,301,2,1,2,11996,True,B-B,Identical,0.034002,-4.149329,-0.552039


In [61]:
df_fin = df_simu.groupby(['cong','test'])[['corr_fin','omean_fin','omax_fin']].mean().reset_index()
df_fin['dif'] = df_fin['corr_fin'] - df_fin['omax_fin']
df_fin

,cong,test,corr_fin,omean_fin,omax_fin,dif
0,Identical,1,-0.648979,-2.509888,-0.519877,-0.129102
1,Identical,2,-0.600525,-2.416174,-0.278329,-0.322195
2,Reversed,1,-0.644314,-2.498674,-0.521211,-0.123103
3,Reversed,2,-0.619701,-2.385869,-0.284518,-0.335184


### PLOT

In [62]:
df_simu.head(24)

,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct,cond,cong,corr_fin,omean_fin,omax_fin
0,0,210,-1,327.0,1830.0,0.797872,1,CABBAGE,327,1,1,1,11,True,F-F,Identical,-0.338427,-0.893156,-0.417645
1,0,1251,-1,-2.0,-2.0,0.627754,2,SCOTCH,819,2,1,1,8,False,B-B,Identical,-0.484410,-1.256948,-0.488696
2,0,145,-1,-2.0,-2.0,0.554538,3,BODY,1198,1,1,1,7,False,F-F,Identical,-0.446878,-1.460890,-0.498679
3,0,1059,-1,-2.0,-2.0,0.638699,4,PICKLE,1391,2,1,1,5,False,B-B,Identical,-0.464744,-1.659222,-0.479372
4,0,711,-1,1198.0,2360.0,0.641787,5,HOE,1519,1,1,1,9,False,F-B,Reversed,-0.455372,-1.882418,-0.477689
5,0,838,-1,-2.0,-2.0,0.527408,6,LOCK,140,2,1,1,1,False,B-F,Reversed,-0.537589,-2.277589,-0.529091
6,0,153,-1,-2.0,-2.0,0.596206,7,BOTTLE,736,1,1,1,6,False,F-B,Reversed,-0.474879,-2.495031,-0.505461
7,0,1587,-1,-2.0,-2.0,0.575857,8,WARDROBE,1240,1,1,1,2,False,F-B,Reversed,-0.481294,-2.719813,-0.495151
8,0,856,-1,-2.0,-2.0,0.545230,9,MAID,607,2,1,1,10,False,B-B,Identical,-0.469664,-2.927994,-0.531170
9,0,702,-1,-2.0,-2.0,0.572660,10,HELMET,1132,2,1,1,3,False,B-F,Reversed,-0.496838,-3.151359,-0.511503


In [63]:
dfgp_test_order = df_simu.groupby(['rep','test','order']).correct.mean().to_frame(name='corr_rate').reset_index()
dfgp_test_order

,rep,test,order,corr_rate
0,1,1,1,0.371667
1,1,1,2,0.370500
2,1,2,1,0.367833
3,1,2,2,0.364833


In [64]:
def contingency_tab(x):
    tmp_x = pd.pivot_table(x,index='pair_idx',columns='test', values= 'correct')
    test1 = pd.Categorical(tmp_x[1], categories=[1, 0])
    test2 = pd.Categorical(tmp_x[2], categories=[1, 0])
    return pd.crosstab(index=test2,columns=test1)

In [65]:
dfgp_rep_cong = df_simu.groupby(['cong']).apply(contingency_tab)
dfgp_rep_cong

col_0               1     0
cong      row_0            
Identical 1      1766   318
          0       470  3446
Reversed  1      1783   529
          0       434  3254

In [66]:
dfgp_rep_cong / 6000

col_0                   1         0
cong      row_0                    
Identical 1      0.294333  0.053000
          0      0.078333  0.574333
Reversed  1      0.297167  0.088167
          0      0.072333  0.542333

In [67]:
df_simu.groupby(['cong','test']).correct.mean()

cong       test
Identical  1       0.372667
           2       0.347333
Reversed   1       0.369500
           2       0.385333
Name: correct, dtype: float64

In [68]:
def Yule_Q(A, B, C, D):
    return (A * D - B * C) / (A * D + B * C)

In [69]:
tab_identical = dfgp_rep_cong.loc['Identical']
Yule_Q(tab_identical[1][1]+0.5,tab_identical[0][1]+0.5,tab_identical[1][0]+0.5,tab_identical[0][0]+0.5)

0.9519551356731978

In [70]:
tab_reversed = dfgp_rep_cong.loc['Reversed']
Yule_Q(tab_reversed[1][1]+0.5,tab_reversed[0][1]+0.5,tab_reversed[1][0]+0.5,tab_reversed[0][0]+0.5)

0.9237487941150078

In [71]:
Yule_Q(0.319, 0.012, 0.006, 0.663)

0.9993193709853524

In [72]:
Yule_Q(0.293, 0.122, 0.049, 0.537)

0.9267935757627709